## Decision Trees (for Classification)

In a decision tree the data is split at each node according to a decision rule. This corresponds to nested _if-then-else_-rules. In the _if_-part of such a rule are decison is made based on a feature of the data record. 

We will use the scikit learn implementation. For this implementation the features must be binary or have (at least) ordinal characteristic. If a feature is e.g. nominal with many values, it must be converted to a set of binary (one-hot-coded) features.


The splitting rules in the scikit learn implementation are binary and are based on a threshold, e.g.
  - _if $x_6 <= 2.14$ then_ left subbranch, _else_ right subbranch.         
  - binary features must be coded as 0/1, so the rule becomes: if $x_2 <= 0.5$ _then_ left subbranch, _else_ right subbranch. 


<!--The structure of the tree will be determined by data (see below) and a training procedure.-->

In the leafs of the tree the (class) predictions are made. There are two possibilities for such an inference: 
   - hard assignment: Predict for the data records which end up on a leaf by the majority class of the training data that end up on that leaf.          
   - soft assignment: Assign probabilities according to the distribution of the classes in respect to the training data which end up on that leaf.

As an example of a decision tree we will learn the following tree from the titanic data set: 
<img src="./data/titanic.png" width="1000px"/>

A full explanation of the tree will be given later. Here just look at the desion rules (first line of the inner nodes) and at the last line of the leafs. In each leaf you see a array with counts of the different targets (train data): [number_died number_survivors] .

### Learning 

Finding a tree that splits the traing data optimal is np-hard. Therefore often a _greedy_-strategy is used:

To build up a decision tree the algorithm starts at the root of the tree. The feature and the threshold 
that splits the training data best (with respect to the classes) are choosen. In an iterative way the whole tree is build up by such splitting rules. 

There are different criteria for measuring the "separation (split) quality". The most important ones are:

- Gini Impurity 
- Information Gain 

In this tutorial we use the information gain.

#### Information Gain as splitting criterion

The entropy with respect to the target class variable $y$ of a training data set $\mathcal D$ is defined as:

$$
 H(y, \mathcal D) = - \sum_{y \in \mathcal Y} p(y|\mathcal D) \log_2 p(y|\mathcal D)
$$
with the domain of the target values $\mathcal Y = \{t_1, t_2,... \}$.


The probabilities are estimated by 
$$
  p(y=t_i, \mathcal D) = |\mathcal D^{(y=t_i)}| /|\mathcal D| 
$$    


with the number of training data $|\mathcal D|$  and the number of training data $|\mathcal D^{(y=t_i)}|$ with target label $t_i$: 


On a node a (binary) split on a feature $x_k$ is made by the split rule $x_k \leq v$. 
As result there are two data sets $\mathcal D_0$ and $\mathcal D_1$ for the left resp. the right branch.

The feature $x_k$ and the split value $v$ are choosen that they maximize the 'reduction of the entropy' measured by the information gain $I$:
$$
  I(y; x_k) = H(y, \mathcal D) - H(y|x_k) = H(y, \mathcal D) - \sum_{j=0}^1 p_jH(y, \mathcal D_j) =
  H(y, \mathcal D) + \sum_{j=0}^1  \sum_{y \in \mathcal Y} \frac{|\mathcal D_j|}{|\mathcal D|} p(y|\mathcal D_j) \log_2 p(y|\mathcal D_j)
$$
Note that $p_{j=0}$  is the estimated probability that a random data record of $\mathcal D$ has feature value $x_k \leq v$ which can be estimated by ${|\mathcal D_0|}/{|\mathcal D|}$ (analog for $j=1$).

$p(y=t_i|\mathcal D_0)$ can also be estimated by the fraction of the counts ${|\mathcal D_0^{(y=t_i)}|}/{|\mathcal D_0|}$. 
So the information gain can be computed just with counts:


$$
  I(y; x_k) = 
   \sum_{y \in \mathcal Y} \frac{|\mathcal D^{(y=t_i)}|}{|\mathcal D|}  \log_2 \frac{|\mathcal D^{(y=t_i)}|}{|\mathcal D|} + \sum_{j=0}^1  \sum_{y \in \mathcal Y} \frac{|\mathcal D_j^{(y=t_i)}|}{|\mathcal D|}  \log_2 \frac{|\mathcal D_j^{(y=t_i)}|}{|\mathcal D_j|}
$$


<!--$|\mathcal D_0|$ respectivly $|\mathcal D_1|$ is the number of elements in the splitted data sets.-->
 

#### Overfitting

Deep decision trees generalize often poorly. The following remedies reduce overfitting: 

- Limitation of the maximal depth of the tree. 
- Pruning with an validation set either during training (pre-pruning) or after training (post-pruning).
- Dimensionality reduction (reducing the number of features before training)


Also often combining decision trees to an ensemble (decision forests) is used against overfitting.  

### Example: Survival of the Titanic 


First you have read in the titanic data with pandas:

In [36]:
import pandas as pd
import numpy as np
train_df = pd.read_csv("titanic-train.csv")

`train_df` is a [_pandas_](http://pandas.pydata.org/) [data frame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html). 
Let's view the data. 

In [37]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [38]:
#train_df.shape

In [39]:
#train_df.dtypes

In [40]:
# Columns with categorical data
categorical = train_df.dtypes[train_df.dtypes=='object'].index
print(train_df[categorical].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
Name        891 non-null object
Sex         891 non-null object
Ticket      891 non-null object
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: object(5)
memory usage: 34.9+ KB
None


Scikit's learn decision trees can handle only numeric data. So we must convert the nominal `Sex` feature. 

In [41]:
train_df["Sex"].head()

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [42]:
train_df["Sex"] = np.where(train_df["Sex"].str.contains("female"),1,0) #female=1, male=0

In [43]:
train_df.Sex.head()

0    0
1    1
2    1
3    1
4    0
Name: Sex, dtype: int32

`Survived` is the target, that we want to predict from the values of the other columns.   
But not all of the other columns are helpful for classification. So we choose a feature set by hand and convert the features into a numpy array for scikit learn. 

In [44]:
y = targets = labels  =  train_df.Survived

columns =  ["Fare", "Pclass", "Sex", "Age", "SibSp"]

train_df[columns].head()


,Fare,Pclass,Sex,Age,SibSp
0,7.2500,3,0,22.0,1
1,71.2833,1,1,38.0,1
2,7.9250,3,1,26.0,0
3,53.1000,1,1,35.0,1
4,8.0500,3,0,35.0,0


In [45]:
features = train_df.as_matrix(columns=columns) # or .val
features

array([[  7.25  ,   3.    ,   0.    ,  22.    ,   1.    ],
       [ 71.2833,   1.    ,   1.    ,  38.    ,   1.    ],
       [  7.925 ,   3.    ,   1.    ,  26.    ,   0.    ],
       ..., 
       [ 23.45  ,   3.    ,   1.    ,      nan,   1.    ],
       [ 30.    ,   1.    ,   0.    ,  26.    ,   0.    ],
       [  7.75  ,   3.    ,   0.    ,  32.    ,   0.    ]])

In [46]:
train_df.loc[:, train_df.isnull().any()].columns.values #Für uns nur Age interessant, 4. Spalte in features

array(['Age', 'Cabin', 'Embarked'], dtype=object)

In [47]:
train_df.Age.mean()

29.69911764705882

There are missing values (`nan`). We use the scikit learn `Imputer` to replace them by the mean of the columns.

In [48]:
from sklearn.preprocessing import Imputer

mean_imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
mean_imputer = mean_imputer.fit(features)
features = mean_imputer.transform(features)
features

array([[  7.25      ,   3.        ,   0.        ,  22.        ,   1.        ],
       [ 71.2833    ,   1.        ,   1.        ,  38.        ,   1.        ],
       [  7.925     ,   3.        ,   1.        ,  26.        ,   0.        ],
       ..., 
       [ 23.45      ,   3.        ,   1.        ,  29.69911765,   1.        ],
       [ 30.        ,   1.        ,   0.        ,  26.        ,   0.        ],
       [  7.75      ,   3.        ,   0.        ,  32.        ,   0.        ]])

Now we are ready to learn a decision tree by the criterion 'Information Gain' and we restrict the depth of the tree to 3.
We use the [scikit learn decison tree module](http://scikit-learn.org/stable/modules/tree.html).

We use a splitting criterion the 'Information Gain'!

In [49]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion="entropy", max_depth=3)
target = y
tree_clf = clf.fit(features, target)
print(tree_clf.score(features, target))

0.822671156004


In [50]:
#"Fare", "Pclass", "Sex", "Age", "SibSp"
tree_clf.feature_importances_

array([ 0.12330431,  0.18665493,  0.5670424 ,  0.09423074,  0.02876762])

`clf` is an instance of a trained decision tree classifier.

The decision tree can be visualized. For this we must write an graphviz dot-File  

In [52]:
from sklearn.externals.six import StringIO


with open("data/titanic.dot", 'w') as f:
    f = tree.export_graphviz(clf, out_file=f, feature_names=columns)



In [55]:
import collections
import pydotplus
dot_data = tree.export_graphviz(clf,
                                feature_names=columns,
                                out_file=None,
                                rounded=True,
                               filled=True)

    
import graphviz
src = graphviz.Source(dot_data, format="png")
src.render("data/titanic_")

'data/titanic_.png'

The dot file can be converted with the graphiz `dot`- renderer to an image.

    dot -Tpng titanic.dot -o titanic.png

Here is the graph:

<img src="./data/titanic.png" width="1000px"/>


Der Classifier berücksichtigt den Target ("Survived", "Not Survived"): 
- 891 Passagiere insgesamt werden anhand der übergebenen Features analysiert
    - 549 sind gestorben
    - 342 überlebten
Der Classifiert ermittelt an dieser Stelle die "beste" Feature (Man Criterion), um den Weg nach unten zu laufen: Sex. Die Trennlinie bei x[Sex] liegt bei 0.5 (da die Werte entweder 0 oder 1 sind). 

Es werden 577 Männer im ganzen Schiff und 314 Frauen ermittelt.

Linker Teilbaum (Men):

_Neues Hauptkriterion ist Fare_:
- Unter Berücksichtigung der Männer (577) als Samples ergibt sich das Fare (Ticketkosten) als main criterion und eine Trennlinie bei 26.269
    - 468 Männer haben weniger als 26.269 ausgegeben (arme Männer)
    - 109 Männer haben mehr als 26.269 ausgegeben (reiche Männer)
    - --> Grafica uomini poveri e ricchi
_Neues Hauptkriterion bzgl. Fare ist Alter_ und Trennlinie bei 13,5:
Von den 577 Männer, wie viele waren arm und jünger als 13,5? 415 Männer







<img src="./data/titanic_.png" width="1200px"/>

According to the decision tree the main criterion (root node) for survival is the sex of the passenger. In the left subtree are the male passengers (sex = 0), in the right subtree the female (sex=1). 

In the leafs the class information is given by a `value` array. Here the second value is the number of survivers in the leafs.

For example the leftmost leaf represents passengers that are male (sex=0) with fare<=26.2687 and age<=13.5. 13 of such boys survived and 2 of them died.

The entropy $- \sum p_i \log_2 (p_i)$ is displayed also at each node (splitting criterion).


### Exercises: Splitting criterion entropy / information gain

### Questions about the dataset

In [56]:
samples = passengers = 891
print("Total passengers: ", samples)

Total passengers:  891


In [57]:
#How many survivors? How many dead people?
#Values in the root node 
dead, survived = train_df.Survived.value_counts()
print("Total dead people: ", dead)
print("Total survived people: ", survived)

Total dead people:  549
Total survived people:  342


In [58]:
#How many men? How many women?
#Value of samples in the first child nodes applied on the total samples of the root
tot_men, tot_women = train_df.Sex.value_counts() #0: 577, 1: 349
print("Total number of men: ", tot_men)
print("Total number of women: ", tot_women)

Total number of men:  577
Total number of women:  314


## Link part of the tree (Men)

In [59]:
#How many men survived? 
stats_men = train_df.Survived[train_df.Sex == 0].value_counts()
men = train_df.Survived[train_df.Sex == 0]
#Value in second node in the tree - left part
print("Men survived: ", stats_men[1])
print("Men dead: ", stats_men[0])

Men survived:  109
Men dead:  468


In [60]:
#How many rich men? How many poor men?
rich_men = train_df.Survived[(train_df.Sex == 0) & (train_df.Fare > 26.269)]
print("Rich men on board: ",len(rich_men))
print(rich_men.value_counts())
print("")
#samples in the third child node on the left side
poor_men = train_df.Survived[(train_df.Sex == 0) & (train_df.Fare <= 26.269)]
print("Poor men on board: ", len(poor_men))
print(poor_men.value_counts())


Rich men on board:  162
0    107
1     55
Name: Survived, dtype: int64

Poor men on board:  415
0    361
1     54
Name: Survived, dtype: int64


In [61]:
#How many young and poor men died?
poor_young_men = train_df.Survived[(train_df.Sex == 0) & (train_df.Fare <= 26.269) & (train_df.Age <= 13.5)]
print("Total - Poor young men: ", len(poor_young_men))

stats_poor_young_men = poor_young_men.value_counts()
print(stats_poor_young_men)
print("Poor young died men: ", stats_poor_young_men[0])
print("Poor young survived men: ", stats_poor_young_men[1])

Total - Poor young men:  15
1    13
0     2
Name: Survived, dtype: int64
Poor young died men:  2
Poor young survived men:  13


### Right part of the tree

In [62]:
#How many women survived? 
stats_women = train_df.Survived[train_df.Sex == 1].value_counts()
#Value in second node in the tree - right side
print("Women survived: ", stats_women[1])
print("Women dead: ", stats_women[0])

Women survived:  233
Women dead:  81


#### Exercise 1: 

Compute the root node entropy (with numpy).

In [63]:
def compute_probs(X):
    return [np.mean(X == c) for c in set(X)]

In [64]:
def entropy(rate):
    return -1*np.sum(rate * np.log2(rate))


In [65]:
def conditional_entropy(p, cond_p):
    result = 0
    result -= np.sum(p*cond_p*np.log2(cond_p))
    return result

In [66]:
print("Total passengers: ", len(train_df))
men, women = train_df.Sex.value_counts()
print("Total men on board: ", men)
print("Total women on board: ", women)

Total passengers:  891
Total men on board:  577
Total women on board:  314


In [67]:
survival_rate = train_df.Survived.value_counts().values
survival_rate

array([549, 342], dtype=int64)

In [68]:
survival_rate_P = train_df.Survived.value_counts(normalize=True).values
survival_rate_P

array([ 0.61616162,  0.38383838])

In [69]:
#Entropy at root
entropy_root = entropy(survival_rate_P)
print(entropy_root)

0.960707901876


#### Exercise 2

Compute the information gain of the first split node (root node). Use the entropy values and the number of data records (samples)
from the decision tree image
$\text{Information gain} = \text{(Entropy of distribution before the split)} – \text{(entropy of distribution after it)}$

In [70]:
#Values Survived by Sex
#equivalent to train_df.groupby('Sex').Survived.value_counts()
survived_by_sex = train_df.groupby('Sex')['Survived'].value_counts()
survived_by_sex

Sex  Survived
0    0           468
     1           109
1    1           233
     0            81
Name: Survived, dtype: int64

In [71]:
#Percentage - Sex and survival rate - wrt the total amount of women or men
survived_by_sex_P = train_df.groupby('Sex')['Survived'].value_counts(normalize=True)
#survived_by_sex_P = train_df.Survived[train_df['Sex'] == 1].value_counts(normalize=True)
print(survived_by_sex_P)

Sex  Survived
0    0           0.811092
     1           0.188908
1    1           0.742038
     0           0.257962
Name: Survived, dtype: float64


In [78]:
#Entropies (on the child node of the tree)
men_entropy = entropy(survived_by_sex_P[0])
women_entropy = entropy(survived_by_sex_P[1])

print("H(men): ", men_entropy)
print("H(women): ", women_entropy)


H(men):  0.6991817891208407
H(women):  0.8236550739295191


In [82]:
#Percentage men and women on board
men_freq, women_freq = train_df.Sex.value_counts(normalize=True)

print("Percentage men and women on board: ")
print("Men: " , men_freq)
print("Women: ", women_freq)


Percentage men and women on board: 
Men:  0.64758698092
Women:  0.35241301908


In [85]:
#percentage survived men/women
print("Survival rate men/women: ")
men_surv_rate = survived_by_sex_P[:2]
women_surv_rate = survived_by_sex_P[2:]
print("Men:\n", men_surv_rate)
print("Women:\n", women_surv_rate)

Survival rate men/women: 
Men:
 Sex  Survived
0    0           0.811092
     1           0.188908
Name: Survived, dtype: float64
Women:
 Sex  Survived
1    1           0.742038
     0           0.257962
Name: Survived, dtype: float64


In [86]:
men_cond_entropy = conditional_entropy(men_freq,men_surv_rate)
men_cond_entropy

0.45278102393122904

In [87]:
women_cond_entropy = conditional_entropy(women_freq,women_surv_rate)
women_cond_entropy

0.29026677128380357

In [88]:
total_conditional_entropy = men_cond_entropy + women_cond_entropy
total_conditional_entropy

0.7430477952150326

In [89]:
#Information gain

entropy_root - total_conditional_entropy

0.21766010666061431

#### Exercise 3
Compute the information gain of the following split table:

|  | class 0  | class 1  |  
|---|---|---|
| feature <= v| 2  | 13  |      
| feature > v | 359  |  41 |   

The numbers are the corresponding data records, e.g. there are 13 data records with target class 1 and feature <= v. 

Write a python function that computes the information gain.The data is given by a python array:

In [93]:
data = np.array([[2.,13.],[359., 41.]])

In [94]:
data_df = pd.DataFrame(data, columns=["y=0", "y=1"], index=["x=0", "x=1"])
data_df

,y=0,y=1
x=0,2.0,13.0
x=1,359.0,41.0


In [134]:
def information_gainY(data):
    N = np.sum(data)
    print("Data: ", N)
    #P(Y)
    Px = np.sum(data, axis=1)/N
    print("P(X): ", Px)
    #P(X)
    Py =np.sum(data, axis=0)/N
    print("P(Y): ", Py)
    #conditional probabilities
    cond_probs = data.T/np.sum(data,axis=1) #tr
    print("Cond probs:", cond_probs)
   
    #P(Y|X)
    P_y_x = cond_probs*np.log2(cond_probs)
    #print("P(Y|X): ", P_y_x)
    
    #H(Y|X)
    H_y_x = -np.sum(Px*P_y_x)
    
    #H(Y)
    H_y = -np.sum(Py*np.log2(Py))
    print("Entropy: ",H_y)
    print("Conditional entropy: ",H_y_x)
    #IG
    IG = H_y - H_y_x
    print("Information Gain: ", IG)
    return 

In [137]:
data = np.array([[2.,13.],[359., 41.]])
print(data.T)
print(np.sum(data,axis=1))

[[   2.  359.]
 [  13.   41.]]
[  15.  400.]


In [135]:
information_gainY(data)

Data:  415.0
P(X):  [ 0.03614458  0.96385542]
P(Y):  [ 0.86987952  0.13012048]
Cond probs: [[ 0.13333333  0.8975    ]
 [ 0.86666667  0.1025    ]]
Entropy:  0.557768514612
Conditional entropy:  0.48011063657
Information Gain:  0.0776578780428


### Prediction

To make predictions with scikit learn, we must convert the data in
the same way as we did it with the training data. Then we could use the method:    

    clf.predict(testdata)
    
Note: The depth of the decision tree is a hyperparameter. So the depth should be determined with the help of a 
validation set or by cross validation.    

def information_gainY(data):
    N = np.sum(data)
    print("Data: ", N)
    #P(Y)
    Py = np.sum(data, axis=1)/N
    print("P(Y): ", Py)
    #P(X)
    Px =np.sum(data, axis=0)/N
    print("P(X): ", Px)
    #conditional probabilities
    cond_probs = data/np.sum(data,axis=0) #tr
    print("Cond probs:", cond_probs)
   
    #P(Y|X)
    P_y_x = cond_probs*np.log2(cond_probs)
    #print("P(Y|X): ", P_y_x)
    
    #H(Y|X)
    H_y_x = -np.sum(Px*P_y_x)
    
    #H(Y)
    H_y = -np.sum(Py*np.log2(Py))
    print("Entropy: ",H_y)
    print("Conditional entropy: ",H_y_x)
    #IG
    IG = H_y - H_y_x
    print("Information Gain: ", IG)
    return 

### Literature

- [Cri] [A. Criminisi, J. Shotton, and E. Konukoglu, Decision Forests for Classification, Regression, Density Estimation, Manifold Learning and Semi- Supervised Learning, no. MSR-TR-2011-114, 28 October 2011.](http://research.microsoft.com/pubs/155552/decisionForests_MSR_TR_2011_114.pdf)
